In [1]:
import pandas as pd
import numpy as np

import os
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight

from tqdm.notebook import tqdm
from PIL import Image

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim

In [2]:
df = pd.read_csv('train.csv')
df

,unified_class,class_id,image_name
0,Оленевые,5,3cf4207b958eade893a2f1618cf062b8.JPG
1,Кошки,2,37698901280c871f426d40afe5c373cd.JPG
2,Заяц,0,20e7b30026001cbfe0b5c0ee16c9ff56.JPG
3,Кошки,2,a1bc8ea546206ee8fc0f1836fda9a5c1.JPG
4,Оленевые,5,54eb76914b84db8a0d56f98125abf588.JPG
...,...,...,...
28010,Оленевые,5,07b420b4fe265b4ed918b46435c025d7.JPG
28011,Пантеры,6,2d1c5918357bbdd729bf79085e55d35e.JPG
28012,Заяц,0,1531efa9f8687e390adf780355acd606.JPG
28013,Кабан,1,2b15eaef0ce9b57b6570709f95a4bea4.JPG


In [3]:
len(set(df.class_id))

10

In [4]:
df.drop(columns=['unified_class'], inplace = True)
df

,class_id,image_name
0,5,3cf4207b958eade893a2f1618cf062b8.JPG
1,2,37698901280c871f426d40afe5c373cd.JPG
2,0,20e7b30026001cbfe0b5c0ee16c9ff56.JPG
3,2,a1bc8ea546206ee8fc0f1836fda9a5c1.JPG
4,5,54eb76914b84db8a0d56f98125abf588.JPG
...,...,...
28010,5,07b420b4fe265b4ed918b46435c025d7.JPG
28011,6,2d1c5918357bbdd729bf79085e55d35e.JPG
28012,0,1531efa9f8687e390adf780355acd606.JPG
28013,1,2b15eaef0ce9b57b6570709f95a4bea4.JPG


In [5]:
sample = pd.read_csv('sample_submission.csv')
sample

,image_name,predicted_class
0,cc27b9b56583a615fb8501e352402eb9.JPG,0
1,87872711fe672676fd34a97e997f9c47.JPG,0
2,424aa1aa8eb5bbdd07275f88077bc86c.JPG,0
3,c5537eaa60525efd7bad4a5560607e83.JPG,0
4,e9f15b67ca49453e281b2b4f245eac13.JPG,0
...,...,...
12953,028668e733cd17ec9b9f1c7e2c657b36.JPG,0
12954,eb1f1152941fdfdd50ff9954010e622a.JPG,0
12955,bfd2dde9f4a5753c9f85b2a93bee9c03.JPG,0
12956,2eaf9c794958a93bb9984441fd5d7f61.JPG,0


In [6]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, paths, labels, dir_path=None):
        super().__init__()
        self.paths = paths
        self.dir_path = dir_path
        self.labels = labels
        
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, ind):
        
        # img = torchvision.io.read_image(self.dir_path+self.paths[ind]) / 255
        img = Image.open(self.dir_path+self.paths[ind]).convert('RGB')
        
        transforms = torchvision.transforms.Compose([
            torchvision.transforms.Resize((256, 256)),
            torchvision.transforms.ToTensor()
        ])
        
        img = transforms(img)
        
        return img, self.labels[ind]

In [7]:
dataset = Dataset(
    paths = df['image_name'].tolist(),
    labels = df['class_id'].tolist(),
    dir_path = 'train/'
)

In [8]:
len(dataset)

28015

In [9]:
dataset[0]

(tensor([[[0.9333, 0.9333, 0.9333,  ..., 0.4549, 0.6431, 0.8078],
          [0.9373, 0.9373, 0.9373,  ..., 0.6627, 0.7451, 0.7686],
          [0.9451, 0.9451, 0.9451,  ..., 0.8745, 0.8588, 0.7804],
          ...,
          [0.9686, 0.9686, 0.9686,  ..., 0.9686, 0.9686, 0.9686],
          [0.9647, 0.9647, 0.9647,  ..., 0.9647, 0.9647, 0.9647],
          [0.9608, 0.9608, 0.9608,  ..., 0.9608, 0.9608, 0.9608]],
 
         [[0.9647, 0.9647, 0.9647,  ..., 0.4784, 0.6667, 0.8353],
          [0.9686, 0.9686, 0.9686,  ..., 0.6824, 0.7647, 0.7922],
          [0.9765, 0.9765, 0.9765,  ..., 0.8941, 0.8824, 0.8039],
          ...,
          [0.9843, 0.9843, 0.9843,  ..., 0.9843, 0.9843, 0.9843],
          [0.9804, 0.9804, 0.9804,  ..., 0.9804, 0.9804, 0.9804],
          [0.9765, 0.9765, 0.9765,  ..., 0.9765, 0.9765, 0.9765]],
 
         [[0.9765, 0.9765, 0.9765,  ..., 0.6118, 0.7882, 0.9216],
          [0.9804, 0.9804, 0.9804,  ..., 0.7843, 0.8471, 0.8667],
          [0.9882, 0.9882, 0.9882,  ...,

In [10]:
df.class_id[0]

5

In [11]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

In [12]:
next(iter(dataloader))[0].shape

torch.Size([64, 3, 256, 256])

In [13]:
len(set(df.class_id))

10

In [14]:
model = torchvision.models.resnet18(weights='IMAGENET1K_V1')
#for param in model.parameters():
#    param.require = False
model.fc = nn.Linear(512, 10)
#model

In [15]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [16]:
model = model.to(device)

In [17]:
# weight = compute_class_weight(class_weight="balanced", classes=np.unique(df.class_id), y=df.class_id)
# weight = torch.FloatTensor(weight).to(device)
# weight

tensor([2.9091, 1.3553, 2.8645, 3.5642, 1.1117, 0.2416, 0.6216, 1.4142, 1.5078,
        3.7008], device='cuda:0')

In [18]:
# loss_fn = nn.CrossEntropyLoss(weight = weight)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)
# optimizer = optim.Adam(model.parameters(), lr=1e-3)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # для scheduler
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.8)

In [19]:
len(dataloader)

438

In [20]:
epochs = 10

for epoch in range(epochs):  # loop over the dataset multiple times
    
    running_loss = 0.0
    i=0
    
    for data in tqdm(dataloader):
        i+=1
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        loss = loss_fn(outputs, labels)
        loss.backward()
        
        optimizer.step()
        # scheduler.step()

        # print statistics
        running_loss += loss.item()

        # if i % 219 == 218:    # print every 2000 mini-batches
        #     print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 219:.4f}')
        #     running_loss = 0.0

    print(f'epoch {epoch + 1}, train loss: {running_loss / len(dataloader):.4f}')

print('Finished Training')

  0%|          | 0/438 [00:00<?, ?it/s]

epoch 1, train loss: 0.4239


  0%|          | 0/438 [00:00<?, ?it/s]

epoch 2, train loss: 0.1775


  0%|          | 0/438 [00:00<?, ?it/s]

...

epoch 8, train loss: 0.0516


  0%|          | 0/438 [00:00<?, ?it/s]

epoch 9, train loss: 0.0373


  0%|          | 0/438 [00:00<?, ?it/s]

epoch 10, train loss: 0.0450
Finished Training


In [21]:
class testDataset(torch.utils.data.Dataset):
    def __init__(self, paths, dir_path=None):
        super().__init__()
        self.paths = paths
        self.dir_path = dir_path
        
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, ind):
        
        # img = torchvision.io.read_image(self.dir_path+self.paths[ind]) / 255
        img = Image.open(self.dir_path+self.paths[ind]).convert('RGB')
        
        transforms = torchvision.transforms.Compose([
            torchvision.transforms.Resize((256, 256)),
            torchvision.transforms.ToTensor()
        ])
        
        img = transforms(img)
        
        return img

In [22]:
test = testDataset(
    paths = sample['image_name'].tolist(),
    dir_path = 'test/'
)

In [23]:
test[0]

tensor([[[0.0078, 0.0078, 0.0039,  ..., 0.0000, 0.0000, 0.0000],
         [0.0157, 0.0157, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
         [0.0235, 0.0235, 0.0196,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0314, 0.0118, 0.1176,  ..., 0.0784, 0.0863, 0.1137],
         [0.0275, 0.0196, 0.0706,  ..., 0.0706, 0.0745, 0.0980],
         [0.0235, 0.0275, 0.3020,  ..., 0.0118, 0.0118, 0.0235]],

        [[0.0078, 0.0078, 0.0039,  ..., 0.0000, 0.0000, 0.0000],
         [0.0157, 0.0157, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
         [0.0235, 0.0235, 0.0196,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0314, 0.0118, 0.1176,  ..., 0.0784, 0.0863, 0.1137],
         [0.0275, 0.0196, 0.0706,  ..., 0.0706, 0.0745, 0.0980],
         [0.0235, 0.0275, 0.3020,  ..., 0.0118, 0.0118, 0.0235]],

        [[0.0078, 0.0078, 0.0039,  ..., 0.0000, 0.0000, 0.0000],
         [0.0157, 0.0157, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
         [0.0235, 0.0235, 0.0196,  ..., 0.0000, 0.0000, 0.

In [24]:
testloader = torch.utils.data.DataLoader(test, batch_size=1, shuffle=False)

In [25]:
classes = np.array(list(set(df.class_id)))
classes

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [26]:
model.eval()
with torch.no_grad():
    pred = [''] * len(testloader)
    i=0
    for X in tqdm(testloader):
        X = X.to(torch.float).to(device)
        pred[i] = classes[model(X)[0].argmax(0)]
        i+=1
Counter(pred)

  0%|          | 0/12958 [00:00<?, ?it/s]

Counter({5: 4673,
         6: 2494,
         4: 1308,
         8: 957,
         1: 937,
         7: 863,
         0: 546,
         2: 485,
         9: 361,
         3: 334})

In [27]:
sample['predicted_class'] = pred
sample

,image_name,predicted_class
0,cc27b9b56583a615fb8501e352402eb9.JPG,4
1,87872711fe672676fd34a97e997f9c47.JPG,5
2,424aa1aa8eb5bbdd07275f88077bc86c.JPG,0
3,c5537eaa60525efd7bad4a5560607e83.JPG,1
4,e9f15b67ca49453e281b2b4f245eac13.JPG,6
...,...,...
12953,028668e733cd17ec9b9f1c7e2c657b36.JPG,1
12954,eb1f1152941fdfdd50ff9954010e622a.JPG,4
12955,bfd2dde9f4a5753c9f85b2a93bee9c03.JPG,5
12956,2eaf9c794958a93bb9984441fd5d7f61.JPG,6


In [28]:
sample.to_csv('sub.csv', index=False)